## Police station location data

### Imports

In [1]:
import pandas as pd
import numpy as np
import geopandas
import geopandas.tools
from shapely.geometry import Point
import pickle
import requests

### Collect & Process Data

Data copied and pasted from the police website.

In [2]:
police_stations = [['Central',415-315-2400,'766 Vallejo Street','SFPDCentralStation@sfgov.org'],
['Southern',415-575-6000,'1251 3rd Street','SFPDSouthernStation@sfgov.org'],
['Bayview',415-671-2300,'201 Williams Avenue','SFPDBayviewStation@sfgov.org'],
['Mission',415-558-5400 ,'630 Valencia Street','SFPDMissionStation@sfgov.org'],
['Northern',415-614-3400,'1125 Fillmore Street','SFPDNorthernStation@sfgov.org'],
['Park',415-242-3000,'1899 Waller Street','SFPDParkStation@sfgov.org'],
['Richmond',415-666-8000,'461 6th Avenue','SFPDRichmondStation@sfgov.org'],
['Ingleside',415-404-4000,'1 Sgt. John V. Young Lane','SFPDInglesideStation@sfgov.org'],
['Taraval',415-759-3100,'2345 24th Avenue','SFPDTaravalStation@sfgov.org'],
['Tenderloin',415-345-7300,'301 Eddy Street','SFPDTenderloinStation@sfgov.org']]

In [3]:
police_locations = pd.DataFrame(police_stations)
police_locations.columns = ['district','phone','address','email']
police_locations.drop(['phone','email'], axis=1, inplace=True)
police_locations['address_formatted'] = police_locations['address'].apply(lambda x: x.replace(' ', '+') + ',San+Francisco,+CA')
police_locations

,district,address,address_formatted
0,Central,766 Vallejo Street,"766+Vallejo+Street,San+Francisco,+CA"
1,Southern,1251 3rd Street,"1251+3rd+Street,San+Francisco,+CA"
2,Bayview,201 Williams Avenue,"201+Williams+Avenue,San+Francisco,+CA"
3,Mission,630 Valencia Street,"630+Valencia+Street,San+Francisco,+CA"
4,Northern,1125 Fillmore Street,"1125+Fillmore+Street,San+Francisco,+CA"
5,Park,1899 Waller Street,"1899+Waller+Street,San+Francisco,+CA"
6,Richmond,461 6th Avenue,"461+6th+Avenue,San+Francisco,+CA"
7,Ingleside,1 Sgt. John V. Young Lane,"1+Sgt.+John+V.+Young+Lane,San+Francisco,+CA"
8,Taraval,2345 24th Avenue,"2345+24th+Avenue,San+Francisco,+CA"
9,Tenderloin,301 Eddy Street,"301+Eddy+Street,San+Francisco,+CA"


All I need to do now is run the formatted address strings through the Google Maps API to retrieve the latitude and longitude...

In [4]:
def get_lat_lng(input_addr):
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(input_addr)
    r = requests.get(url)
    r_json = r.json()
    lat = r_json['results'][0]['geometry']['location']['lat']
    lng = r_json['results'][0]['geometry']['location']['lng']
    return Point(lng, lat)

police_locations['geometry'] = police_locations['address_formatted'].apply(get_lat_lng)

In [5]:
police_locations.drop(['address','address_formatted'], axis=1, 
                      inplace=True)

In [6]:
police_locations

,district,geometry
0,Central,POINT (-122.4099186 37.7987322)
1,Southern,POINT (-122.3894121 37.7723802)
2,Bayview,POINT (-122.3979806 37.7297322)
3,Mission,POINT (-122.422005 37.762849)
4,Northern,POINT (-122.4324665 37.7801858)
5,Park,POINT (-122.455287 37.767797)
6,Richmond,POINT (-122.4644668 37.7799276)
7,Ingleside,POINT (-122.4462154 37.7246756)
8,Taraval,POINT (-122.4815003 37.7437335)
9,Tenderloin,POINT (-122.4128962 37.783669)


In [7]:
police_locations = geopandas.GeoDataFrame(police_locations['geometry'], 
                                          geometry='geometry')

### Save Data

In [8]:
pickle.dump(police_locations, open('police_locations.pkl', 'wb'))